In [13]:
from __future__ import absolute_import, division, print_function, unicode_literals

import h5py
import logging
from os import path
from pandas import HDFStore
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
from keras import regularizers
from utils import get_callbacks,get_callbacks_t,windowed_dataset,plot_report,build_mdpi,split_train_test_val
import tensorflow_docs.plots

%matplotlib inline

In [15]:
#Parameters
EPOCHS = 1000
INITIAL_EXAMPLES = 50
FEATURES = 4096
window_size = 128
shuffle_buffer = 40000
coeff = 22

In [16]:
#Load data
coeff = 17

#load source origin hdf5 file
root_path = "Z:/elekin"
rdo_root_path = path.join(root_path,"02-RESULTADOS/03-HANDWRITTING")
h5file = path.join(rdo_root_path, "00-OUTPUT/archimedean-")
h5filename = h5file + str(coeff) + ".h5"
hdf = HDFStore(h5filename)

raw_df = hdf['results/residues/rd'].T
#normalization
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_df = pd.DataFrame(scaler.fit_transform(raw_df))
#get labels
scaled_df['labels'] = hdf.get('source/labels').values
#shuffle dataset
raw_dataset = scaled_df.sample(frac=1, random_state=2020)

In [17]:
#Feature engineering

#preprocess labels to get the right data type for training
labels = raw_dataset["labels"].values.astype('int8')
timeseries = raw_dataset.loc[:, raw_dataset.columns != 'labels'].values.astype("float32")

#data augmentation, build features windows
for i,features in enumerate(timeseries):
    new = windowed_dataset(features,window_size,window_size,labels[i])
    if i>0:
        dataset = tf.data.Dataset.concatenate(dataset,new)
    else:
        dataset = new    

In [18]:
DATASET_SIZE = int(len(list(iter(dataset.cache()))))

In [ ]:
# Very important to guarantee good shuffle data. Get some label values to review shuffle effect
for x,y in dataset.take(15):
    #print("x= ",x)
    print("y= ",y)

In [19]:
# Build datasets splits
train, val, test = split_train_test_val(dataset, DATASET_SIZE, train_ratio=0.7, 
                                                                val_ratio=0.15, test_ratio=0.15, shuffle_buffer=shuffle_buffer)
train_dataset = train.batch(16).prefetch(1)
val_dataset = val.batch(8).prefetch(1)
test_dataset = test.batch(8).prefetch(1)

In [20]:
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)
train_size, val_size, test_size

(138915, 29767, 29767)

In case we want to validate training splits execute cell below

In [ ]:
len(list(iter(train_dataset))),len(list(iter(val_dataset))),len(list(iter(test_dataset)))

In [ ]:
# Very important to guarantee good shuffle data. Get some label values to review shuffle effect
for x,y in train_dataset.take(2):
    #print("x= ",x)
    print("y= ",y)

In [21]:
# Build Neural Network architecture and compile
mdpi = build_mdpi(window_size)# build cnn1d architecture
#mdpi.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-1, momentum=0.9),
mdpi.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=7e-8, epsilon=None, decay=1e-4, beta_1=0.9, beta_2=0.99, amsgrad=False),
                          loss='binary_crossentropy',metrics=['accuracy'])#compile
#mdpi.summary()#review the architecture

In [ ]:
## Training
tf.keras.backend.clear_session()# clean previous training cached data
histories = {}
#histories['mdpi'] = mdpi.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS, callbacks=get_callbacks('mdpi',10), verbose=0, use_multiprocessing=True,)
histories['mdpi/local'] = mdpi.fit(train_dataset.cache(), validation_data=val_dataset.cache(), epochs=EPOCHS, callbacks=get_callbacks_t('mdpi/local',10), verbose=0)


Epoch: 0, accuracy:0.4966,  loss:0.7782,  val_accuracy:0.4781,  val_loss:0.7202,  
..........
Epoch: 10, accuracy:0.5006,  loss:0.7709,  val_accuracy:0.4747,  val_loss:0.7205,  
..........
Epoch: 20, accuracy:0.5019,  loss:0.7672,  val_accuracy:0.4769,  val_loss:0.7206,  
..........
Epoch: 30, accuracy:0.4992,  loss:0.7687,  val_accuracy:0.4780,  val_loss:0.7206,  
..........
Epoch: 40, accuracy:0.5011,  loss:0.7678,  val_accuracy:0.4779,  val_loss:0.7206,  
..........
Epoch: 50, accuracy:0.5009,  loss:0.7674,  val_accuracy:0.4783,  val_loss:0.7206,  
..........
Epoch: 60, accuracy:0.5008,  loss:0.7664,  val_accuracy:0.4782,  val_loss:0.7206,  
..........
Epoch: 70, accuracy:0.4999,  loss:0.7676,  val_accuracy:0.4781,  val_loss:0.7206,  
..........
Epoch: 80, accuracy:0.4991,  loss:0.7676,  val_accuracy:0.4782,  val_loss:0.7206,  
..........
Epoch: 90, accuracy:0.5004,  loss:0.7679,  val_accuracy:0.4785,  val_loss:0.7206,  
..........
Epoch: 100, accuracy:0.4989,  loss:0.7691,  val_ac

In [ ]:
#Evaluation: plot training & validation accuracy values
plot_report(histories['mdpi'])

In [ ]:
print('\n# Evaluate on test data')
results = mdpi.evaluate(test_dataset)
print('test loss, test acc:', results)